In [20]:
from read_data import get_training, get_test, get_Doc2Vec, get_data
from processing import combine_with_vec, exclude_non_numeric
from sklearn.svm import SVC, LinearSVC
import pandas as pd
import numpy as np
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import MinMaxScaler, FunctionTransformer
from sklearn.model_selection import train_test_split
import pickle

In [21]:
# Training data
X, y = get_training()
train_name_vec50, train_ingr_vec50, train_steps_vec50 = get_Doc2Vec(data="train", num_features=50)
train_name_vec100, train_ingr_vec100, train_steps_vec100 = get_Doc2Vec(data="train", num_features=100)

# SVM pipelines
difference in models: 
https://stackoverflow.com/questions/27912872/what-is-the-difference-between-svc-and-svm-in-scikit-learn

In [22]:
# SVC with BRF kernel
steps = [('MinMaxScaler', MinMaxScaler()), ('LogTransformer', FunctionTransformer(np.log1p)), ('SVC-RBF', SVC(kernel='rbf', gamma='auto', C=1.0, random_state=30027))]
SVC_RBF_Doc2Vec_pipeline = Pipeline(steps=steps)

In [23]:
# LinearSVC
steps_linear = [('MinMaxScaler', MinMaxScaler()), ('LogTransformer', FunctionTransformer(np.log1p)), ('LinearSVC', LinearSVC(C=1.0, max_iter=10000))]
LinearSVC_Doc2Vec_pipeline = Pipeline(steps=steps_linear)

### Doc2Vec preparation

In [24]:
# Combine Doc2Vec with 50 features and non-numeric train data
temp_X = X.copy()
temp_X = exclude_non_numeric(temp_X)
temp_train_name_vec50 = train_name_vec50.copy()
temp_train_ingr_vec50 = train_ingr_vec50.copy()
temp_train_steps_vec50 = train_steps_vec50.copy()
X_combined_50: pd.DataFrame = combine_with_vec(temp_X, temp_train_name_vec50, temp_train_ingr_vec50, temp_train_steps_vec50)

In [25]:
X_train_50, X_test_50, y_train_50, y_test_50 = train_test_split(X_combined_50, y, test_size=0.2, random_state=30027)

In [26]:
# Combine Doc2Vec with 50 features and non-numeric train data
temp_X = X.copy()
temp_X = exclude_non_numeric(temp_X)
temp_train_name_vec100 = train_name_vec100.copy()
temp_train_ingr_vec100 = train_ingr_vec100.copy()
temp_train_steps_vec100 = train_steps_vec100.copy()
X_combined_100: pd.DataFrame = combine_with_vec(temp_X, temp_train_name_vec100, temp_train_ingr_vec100, temp_train_steps_vec100)

In [27]:
X_train_100, X_test_100, y_train_100, y_test_100 = train_test_split(X_combined_100, y, test_size=0.2, random_state=30027)

## SVM with RBF kernel and Doc2Vec with 50 features
Warning: takes 5min to run below code

In [ ]:
%%time
partial_SVC_RBF_50Doc2Vec_model = SVC_RBF_Doc2Vec_pipeline.fit(X_train_50, y_train_50)

Above model training time
- CPU times: user 4min 32s, sys: 4.57 s, total: 4min 37s
- Wall time: 4min 49s

In [ ]:
# Score
partial_SVC_RBF_50Doc2Vec_model.score(X_test_50, y_test_50)
# -> 0.7095

### Model with all train data
Warning: Takes long time!!!!!

In [ ]:
%%time
SVC_RBF_50Doc2Vec_model = SVC_RBF_Doc2Vec_pipeline.fit(X_combined_50, y)

Above model training time
- CPU times: user 6min 57s, sys: 6.75 s, total: 7min 3s
- Wall time: 7min 19s

In [ ]:
# save model
pickle.dump(SVC_RBF_50Doc2Vec_model, open("models/SVC-RBF-50Doc2Vec.sav", "wb"))

## SVM with RBF kernel and Doc2Vec with 100 features

Warning: below code take loooong time to excute!!!

In [13]:
%%time
partial_SVC_RBF_100Doc2Vec_model = SVC_RBF_Doc2Vec_pipeline.fit(X_train_100, y_train_100)

CPU times: user 9min 4s, sys: 13.9 s, total: 9min 18s
Wall time: 10min 1s


Above model training time
- CPU times: user 8min 42s, sys: 9.23 s, total: 8min 51s
- Wall time: 9min 15s

In [15]:
# Score
partial_SVC_RBF_100Doc2Vec_model.score(X_test_100, y_test_100)
# -> 0.707375 Score decreased lol

0.707375

### Model with all train data
Warning: take very loooooong time!!!!

In [16]:
%%time
SVC_RBF_100Doc2Vec_model = SVC_RBF_Doc2Vec_pipeline.fit(X_combined_100, y)

CPU times: user 13min 48s, sys: 18.5 s, total: 14min 6s
Wall time: 15min 8s


In [17]:
# save model
pickle.dump(SVC_RBF_100Doc2Vec_model, open("models/SVC-RBF-100Doc2Vec.sav", "wb"))

## LinearSVC with 50 features Doc2Vec 

In [29]:
%%time
partial_LinearSVC_50Doc2Vec_model = LinearSVC_Doc2Vec_pipeline.fit(X_train_50, y_train_50)

CPU times: user 51.9 s, sys: 1.82 s, total: 53.8 s
Wall time: 1min 1s


In [30]:
partial_LinearSVC_50Doc2Vec_model.score(X_test_50, y_test_50)
# -> 0.7205

0.7205

In [31]:
%%time
LinearSVC_50Doc2Vec_model = LinearSVC_Doc2Vec_pipeline.fit(X_combined_50, y)

CPU times: user 1min 1s, sys: 1.75 s, total: 1min 2s
Wall time: 1min 9s


In [32]:
pickle.dump(LinearSVC_50Doc2Vec_model, open("models/LinearSVC_50Doc2Vec_model.sav", "wb"))